In [12]:
import arcpy
import os

In [13]:
parcels = r"C:\EsriTraining\PYTS\Data\CountyData.gdb\ParcelPts"
arpxFile = r"C:\EsriTraining\PYTS\PYTSClass\PYTSClass.aprx"
exp = '"PriceSquFt" <= 90'

## Get the Parcels that meet the criteria

In [14]:
uFields = ['SquFoot', 'TaxValue', 'PriceSquFt']
with arcpy.da.UpdateCursor(parcels, uFields) as uCursor:
    for row in uCursor:
        row[2] = row[1] / row[0]
        uCursor.updateRow(row)
del uCursor

parcelLyr = arcpy.management.SelectLayerByAttribute(in_layer_or_view=parcels, selection_type='NEW_SELECTION', where_clause=exp).getOutput(0) 
parcelLyr

## Get the ArcGIS Pro Project

In [18]:
aprx = arcpy.mp.ArcGISProject(arpxFile)
pagx = r"C:\EsriTraining\PYTS\PYTSClass\BaseLayout.pagx"
mapName = arcpy.Describe(parcelLyr).baseName + " Map" # name the map after the input layer
proMap = aprx.createMap(name=mapName, map_type="Map")
proMap.addLayer(parcelLyr)

## Create the Layout from PAGX

In [29]:
#parcelLayout = aprx.createLayout(page_width=10, page_height=10, page_units="inch", name="test")
newLayout = aprx.importDocument(pagx)
lyt = aprx.listLayouts()[0]

In [53]:
[field.name for field in arcpy.Describe(parcelLyr).fields]

['OBJECTID',
 'Shape',
 'Parcel_ID',
 'Street_Number',
 'Street_Name',
 'City',
 'State',
 'Zip',
 'Owner_Name',
 'Phone_Number',
 'SquFoot',
 'TaxValue',
 'PriceSquFt']

In [35]:
help(lyt.createSpatialMapSeries)

Help on method createSpatialMapSeries in module arcpy._mp:

createSpatialMapSeries(mapframe, index_layer, name_field, sort_field=None) method of arcpy._mp.Layout instance
    Layout.createSpatialMapSeries(mapframe, index_layer, name_field,
    {sort_field})
    
    
    This method creates a spatial MapSeries object for a layout.
    
    
    mapframe(MapFrame):
    A MapFrame object that has an  associated map that includes the index
    layer needed for creating the spatial map series pages.
    
    
    index_layer(Layer):
    The index Layer object needed for the extents and attribute information
    for  each page. The default value is None.
    
    
    name_field(String):
    The field name of the field used to control the name of each page in the
    map series.  Ideally, all field values are unique so each page can be
    uniquely identified.
    
    
    sort_field{String}:
    The name of the field that will be used to sort the map series pages.



In [55]:
mapSeries = lyt.createSpatialMapSeries(mapframe=lyt.listElements("mapframe_element")[0],index_layer=parcelLyr, name_field="Parcel_ID")

ValueError: Layout: index_layer is not associated with the layout.

In [69]:
lyt.listElements("mapframe_element")[0].map

'Map1'

## Populate the layout 

In [7]:
polyExtent = arcpy.Describe(parcelLyr).extent.polygon

parcelFrame = parcelLayout.createMapFrame(geometry=polyExtent, map=proMap)

#parcelLayout.

## Export the layout

In [8]:
outPDF = f'C:\EsriTraining\PYTS\PYTSClass\{parcelLayout.name}.pdf'
aprx.save()
parcelLayout.exportToPDF(outPDF, 100, 
                    image_quality = 3,
                    output_as_image = True)

'C:\\EsriTraining\\PYTS\\PYTSClass\\test.pdf'

In [9]:
proMap.name

'ParcelPts Map'

In [11]:
%reset -f